In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import colors
from matplotlib import ticker
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import rcParams
from matplotlib import patches
import scipy.io
import astropy.constants as const
import juanfit
import copy
from scipy import interpolate
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.integrate import simps
from scipy.signal import find_peaks
rcParams['axes.linewidth'] = 1.5
import cmcrameri.cm as cmcm
import xarray as xr

In [4]:
awsom_data_set = scipy.io.readsav(r'../../../EIS_2021/save/spectrum/box/box_2063_awsomr_100k.sav',verbose = True,python_dict=True)

--------------------------------------------------
Date: Tue Jan 25 11:49:49 2022
User: yjzhu
Host: ptolemy.engin.umich.edu
--------------------------------------------------
Format: 12
Architecture: x86_64
Operating System: linux
IDL Version: 8.7.2
--------------------------------------------------
Successfully read 6 records of which:
 - 1 are of type TIMESTAMP
 - 3 are of type VARIABLE
 - 1 are of type VERSION
--------------------------------------------------
Available variables:
 - x [<class 'numpy.ndarray'>]
 - w [<class 'numpy.ndarray'>]
 - param [<class 'numpy.ndarray'>]
--------------------------------------------------


In [5]:
m_p = const.m_p.cgs.value
k_b = const.k_B.cgs.value
c = const.c.cgs.value
rs = const.R_sun.cgs.value

In [6]:
p_e_ratio = 0.83
awsom_x = awsom_data_set['x'][0,0,0,:]
awsom_y = awsom_data_set['x'][1,0,:,0]
awsom_z = awsom_data_set['x'][2,:,0,0]
awsom_x_grid = awsom_data_set['x'][0,:,:,:]
awsom_y_grid = awsom_data_set['x'][1,:,:,:]
awsom_z_grid = awsom_data_set['x'][2,:,:,:]
rho = awsom_data_set['w'][0,:,:,:]
n = rho/m_p
#n = np.nan_to_num(n,nan=0)
ux = awsom_data_set['w'][1,:,:,:]
uy = awsom_data_set['w'][2,:,:,:]
uz = awsom_data_set['w'][3,:,:,:]
bx = awsom_data_set['w'][4,:,:,:]
by = awsom_data_set['w'][5,:,:,:]
bz = awsom_data_set['w'][6,:,:,:]
I01 = awsom_data_set['w'][8,:,:,:]
I02 = awsom_data_set['w'][9,:,:,:]
p = awsom_data_set['w'][11,:,:,:]
t = p/n/k_b
#t = np.nan_to_num(t,nan=1e3)
p_e = awsom_data_set['w'][10,:,:,:]
t_e = p_e/n/k_b
t_e = np.nan_to_num(t_e,nan=1e3)
n = np.nan_to_num(n,nan=1)
n = n + 1
b_tot = np.sqrt(np.square(bx) + np.square(by) + np.square(bz))
u_tot = np.sqrt(np.square(ux) + np.square(uy) + np.square(uz))
bx_rot = np.zeros_like(bx)
by_rot = np.zeros_like(by)
bz_rot = np.zeros_like(bz)
ux_rot = np.zeros_like(ux)
uy_rot = np.zeros_like(uy)
uz_rot = np.zeros_like(uz)

rot_DD = np.resize(awsom_data_set["param"],(3,3))
for ii in range(ux.shape[0]):
    for jj in range(ux.shape[1]):
        for kk in range(ux.shape[2]):
            bx_rot[ii,jj,kk],by_rot[ii,jj,kk],bz_rot[ii,jj,kk] = np.matmul([bx[ii,jj,kk],by[ii,jj,kk],bz[ii,jj,kk]],rot_DD.T)
            ux_rot[ii,jj,kk],uy_rot[ii,jj,kk],uz_rot[ii,jj,kk] = np.matmul([ux[ii,jj,kk],uy[ii,jj,kk],uz[ii,jj,kk]],rot_DD.T)

/var/folders/9p/kj06pc4s4m30vcklbzw2hhgw0000gn/T/ipykernel_39416/2558167840.py:20: RuntimeWarning: invalid value encountered in divide
  t = p/n/k_b
/var/folders/9p/kj06pc4s4m30vcklbzw2hhgw0000gn/T/ipykernel_39416/2558167840.py:23: RuntimeWarning: invalid value encountered in divide
  t_e = p_e/n/k_b


In [7]:
chianti_emiss_tbl = scipy.io.readsav("../../sav/AWSoM/chianti_table/FeXIV_FeX_emiss.sav",verbose=True,python_dict=True)

--------------------------------------------------
Date: Mon Oct 24 01:08:15 2022
User: yjzhu
Host: Yingjies-MacBook-Pro.local
--------------------------------------------------
Format: 11
Architecture: x86_64
Operating System: darwin
IDL Version: 8.2
--------------------------------------------------
Successfully read 8 records of which:
 - 1 are of type TIMESTAMP
 - 5 are of type VARIABLE
 - 1 are of type VERSION
--------------------------------------------------
Available variables:
 - fexiv_emiss_array [<class 'numpy.ndarray'>]
 - fex_emiss_array [<class 'numpy.ndarray'>]
 - temp [<class 'numpy.ndarray'>]
 - dens [<class 'numpy.ndarray'>]
 - height [<class 'numpy.ndarray'>]
--------------------------------------------------


In [8]:
chianti_emiss_tbl["fexiv_emiss_array"].shape

(11, 8, 100)

In [9]:
FeXIV_emiss = xr.DataArray(data=chianti_emiss_tbl["fexiv_emiss_array"],
            coords=[chianti_emiss_tbl["height"],chianti_emiss_tbl["dens"],chianti_emiss_tbl["temp"]],
            dims = ["height","dens","temp"])

In [10]:
FeXIV_emiss

<xarray.DataArray (height: 11, dens: 8, temp: 100)>
array([[[3.17281801e-18, 3.17281801e-18, 3.17281801e-18, ...,
         3.17281801e-18, 3.17281801e-18, 3.17281801e-18],
        [3.17281801e-18, 3.17281801e-18, 3.17281801e-18, ...,
         3.17281801e-18, 3.17281801e-18, 3.17281801e-18],
        [3.17281801e-18, 3.17281801e-18, 3.17281801e-18, ...,
         3.17281801e-18, 3.17281801e-18, 3.17281801e-18],
        ...,
        [3.17281801e-18, 3.17281801e-18, 3.17281801e-18, ...,
         3.17281801e-18, 3.17281801e-18, 3.17281801e-18],
        [3.17281801e-18, 3.17281801e-18, 3.17281801e-18, ...,
         3.17281801e-18, 3.17281801e-18, 3.17281801e-18],
        [3.17281801e-18, 3.17281801e-18, 3.17281801e-18, ...,
         3.17281801e-18, 3.17281801e-18, 3.17281801e-18]],

       [[3.18994078e-18, 3.18994078e-18, 3.18994078e-18, ...,
         3.18994078e-18, 3.18994078e-18, 3.18994078e-18],
        [3.18994078e-18, 3.18994078e-18, 3.18994078e-18, ...,
         3.18994078e-18, 3.18994078e-18, 3.18994078e-18],
        [3.18994078e-18, 3.18994078e-18, 3.18994078e-18, ...,
         3.18994078e-18, 3.18994078e-18, 3.18994078e-18],
...
        [3.19547994e-18, 3.19547994e-18, 3.19547994e-18, ...,
         3.19547994e-18, 3.19547994e-18, 3.19547994e-18],
        [3.19547994e-18, 3.19547994e-18, 3.19547994e-18, ...,
         3.19547994e-18, 3.19547994e-18, 3.19547994e-18],
        [3.19547994e-18, 3.19547994e-18, 3.19547994e-18, ...,
         3.19547994e-18, 3.19547994e-18, 3.19547994e-18]],

       [[3.19554255e-18, 3.19554255e-18, 3.19554255e-18, ...,
         3.19554255e-18, 3.19554255e-18, 3.19554255e-18],
        [3.19554255e-18, 3.19554255e-18, 3.19554255e-18, ...,
         3.19554255e-18, 3.19554255e-18, 3.19554255e-18],
        [3.19554255e-18, 3.19554255e-18, 3.19554255e-18, ...,
         3.19554255e-18, 3.19554255e-18, 3.19554255e-18],
        ...,
        [3.19554255e-18, 3.19554255e-18, 3.19554255e-18, ...,
         3.19554255e-18, 3.19554255e-18, 3.19554255e-18],
        [3.19554255e-18, 3.19554255e-18, 3.19554255e-18, ...,
         3.19554255e-18, 3.19554255e-18, 3.19554255e-18],
        [3.19554255e-18, 3.19554255e-18, 3.19554255e-18, ...,
         3.19554255e-18, 3.19554255e-18, 3.19554255e-18]]])
Coordinates:
  * height   (height) >f8 1.0 1.5 2.0 2.5 3.0 3.5 4.0 4.5 5.0 5.5 6.0
  * dens     (dens) >f4 6.0 7.0 8.0 9.0 10.0 11.0 12.0 13.0
  * temp     (temp) >f4 4.0 4.05 4.1 4.15 4.2 4.25 ... 8.75 8.8 8.85 8.9 8.95

In [16]:
FeXIV_emiss.interp(dens=10,temp=6,height=1.0)

<xarray.DataArray ()>
array(3.17281801e-18)
Coordinates:
    dens     int64 10
    temp     int64 6
    height   float64 1.0